In [89]:
import numpy as np
import os, time
import cv2
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model

In [90]:
validPicPath = 'members/'

valid = "0.jpg"

compares = ["trump.jpg", "obama.jpg" ]

# OpenCV Cascade classifier detect face
cascade_path = r'C:\Users\root\PycharmProjects\Webcam\model\haarcascade_frontalface_alt2.xml'

# Facenet model need 160×160 image size
image_size = 160

# MS-Celeb-1M dataset pretrained Keras model
model_path = 'model/facenet_keras.h5'

model = load_model(model_path)

In [91]:
def prewhiten(x):

    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size

    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size

    else:
        raise ValueError("Dimension should be 3 or 4")

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj

    return y

In [92]:
#L2
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))

    return output

# detect face area, extract face, and resize face img 
def align_image(img, margin):
    cascade = cv2.CascadeClassifier(cascade_path)
    
    faces = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)

    if(len(faces)>0):
        (x, y, w, h) = faces[0]
        face = img[y:y+h, x:x+w]
        faceMargin = np.zeros((h+margin*2, w+margin*2, 3), dtype = "uint8")

        faceMargin[margin:margin+h, margin:margin+w] = face
#         cv2.imwrite(str(time.time())+".jpg", faceMargin)

        aligned = resize(faceMargin, (image_size, image_size), mode='reflect')
#         cv2.imwrite(str(time.time())+"_aligned.jpg", aligned)

        return aligned

    else:
        return None

# img preprocss
def preProcess(img):
    whitenImg = prewhiten(img)
    whitenImg = whitenImg[np.newaxis, :]

    return whitenImg

In [93]:

imgValid = validPicPath + valid

aligned = align_image(cv2.imread(imgValid), 6)

if(aligned is None):

    print("Cannot find any face in image: {}".format(imgValid))

else:

    faceImg = preProcess(aligned)
    #–> model會輸出128維度的臉孔特徵向量，接著我們將它們合併並進行L2正規化。Z

    embs_valid = l2_normalize(np.concatenate(model.predict(faceImg)))

  #同上方的valid圖片，依序取得各圖片人臉的臉孔特徵向量，再與valid進行歐氏距離計算。

    for member in compares:

        img_file = validPicPath + member

        aligned = align_image(cv2.imread(img_file), 6)

        if(aligned is not None):

            faceImg = preProcess(aligned)

            embs = l2_normalize(np.concatenate(model.predict(faceImg)))

            distanceNum = distance.euclidean(embs_valid, embs)

            print("Diff with {} is {}".format(member, distanceNum))

Diff with trump.jpg is 1.4174655675888062
Diff with obama.jpg is 0.6872093081474304
